In [3]:
import pandas as pd
%load_ext autoreload
%autoreload 2
    
import sys, os
sys.path.append(os.path.abspath(".."))
import cga_utils
import prompt_versions_g3


In [4]:
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.s2_1.db"))

In [5]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="gemma3:4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0, num_predict=400)  # vagy bármely más elérhető Ollama modell
#llm = ChatOllama(model="hf.co/unsloth/Qwen3-4B-Instruct-2507-GGUF:latest")  # vagy bármely más elérhető Ollama modell

In [6]:
dataset_tr = pd.read_json('dataset_raw/tatqa_dataset_train.json')    
dataset_t = pd.read_json('dataset_raw/tatqa_dataset_test.json')    
dataset_tg = pd.read_json('dataset_raw/tatqa_dataset_test_gold.json')    
dataset_d = pd.read_json('dataset_raw/tatqa_dataset_dev.json')    

sample_df=dataset_tg
#sample_df = dataset_tg.sample(frac=0.20, random_state=42)
#sample_df = dataset_tr.sample(frac=0.031, random_state=42)


In [7]:
len(dataset_tr), len(dataset_d),  len(dataset_t), len(dataset_tg), len(sample_df)

(2201, 278, 278, 277, 277)

In [8]:
for i, item in sample_df.iterrows():    
    table = item['table']['table']     
    for q_item in item['questions']:
        for idev, itemd in dataset_d.iterrows():
            tabled = itemd['table']['table']     
            for q_itemd in itemd['questions']: 
                if q_itemd['answer_type'] == "arithmetic" and q_item['question'] == q_itemd['question']:
                #if q_item['question'] == q_itemd['question']:
                    print('duplicate', q_item['question'], q_itemd['question'], q_item['answer'],  q_itemd['answer'])
                

duplicate What was the percentage change in total revenue between 2018 and 2019? What was the percentage change in total revenue between 2018 and 2019? 10.98 5.22
duplicate What was the percentage change in total revenue between 2018 and 2019? What was the percentage change in total revenue between 2018 and 2019? 10.98 5.22
duplicate What is the percentage change in revenues from 2018 to 2019? What is the percentage change in revenues from 2018 to 2019? 13.48 1.05
duplicate What is the average voyage revenue in 2018 and 2019? What is the average voyage revenue in 2018 and 2019? 303118 303118


In [28]:
import pandas as pd
messages = prompt_versions_g3.prompt_versions['V18'] #Base
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1771619762.179412


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 5057548.5 good answer:  995684.5  0.005s
<< (948,578+1,042,791)/2 || [4820218.0, 5294879.0, 5057548.5] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.96 good answer:  -9.03  0.004s
<< ((948,578-1,042,791)/1,042,791) * 100 || [5294879.0, 4820218.0, -0.08964529689913595] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018? failure: 643.0 good answer:  643  0.003s
<< 17,845-17,202 || [17845.0, 17202.0, 643.0] || [17845.0, 17202.0] || True >>
cac10d43fde9e07342fa7144876e77e7
What was the increase / (decrease) in wireless capital expenditure from 2018 to 2019? Success: 234.0 0.003s
<< 1,320 - 1,086 || [1320.0, 1086.0, 234.0] || [1320.0, 1086.0] || 

<string>:10: SyntaxWarning: 'generator' object is not subscriptable; perhaps you missed a comma?
<string>:10: SyntaxWarning: 'generator' object is not subscriptable; perhaps you missed a comma?


 failure: 187.0 good answer:  330  0.003s
<< (473+187)/2 || [187.0] || [473.0, 187.0] || False >>
*** Overall EM:  0.46607669616519176 / 200
65a6fe3c82ae4de86cd088dde95e149d
What was the change in deferred value in 2019 from 2018? failure: 26.4 good answer:  0.9  0.003s
<< 40.9-40.0 || [40.9, 14.5, 26.4] || [40.9, 40.0] || False >>
0dd04258827e46504f73f8426322f463
What was the percentage change in deferred value in 2019 from 2018? failure: 0.69 good answer:  2.25  0.003s
<< ((40.9-40.0)/40.0) * 100 || [14.6, 14.5, 0.6896551724137907] || [40.9, 40.0, 40.0] || False >>
40acacac7f638d5cfdff96a4b1c0995d
What is the average year-on-year change in Weighted average grant date fair value from 2017 to 2019?  failure: 1.74 good answer:  15.77  0.002s
<< (((4.72-3.36)/3.36+(3.36-3.69)/3.69)/2) * 100 || [1.7400000000000002] || [4.72, 3.36, 3.36, 3.36, 3.69, 3.69] || False >>
f7e00028b6447875f8005ab8c18639a4
What was the percentage increase / (decrease) in Weighted average grant date fair value fro

In [29]:
annotated_results = cga_utils.annotate_results(res)
annotated_results.to_csv('res/g3_v18_test_v2.csv')
cga_utils.calc_overall_em(annotated_results)

Annotaion warning:  invalid syntax (<unknown>, line 12)
Annotaion warning:  invalid decimal literal (<unknown>, line 12)


0.4394904458598726

In [25]:
import pandas as pd
messages = prompt_versions_g3.prompt_versions['V19d'] #Best alg
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1771619686.4233463


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 5057548.5 good answer:  995684.5  0.005s
<< (948,578+1,042,791)/2 || [4820218.0, 5294879.0, 5057548.5] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.96 good answer:  -9.03  0.004s
<< ((948,578-1,042,791)/1,042,791) * 100 || [5294879.0, 4820218.0, -0.08964529689913595] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018?[Error]unsupported operand type(s) for -: 'tuple' and 'tuple'
[Error]unsupported operand type(s) for -: 'tuple' and 'tuple'
 failure: [Error]unsupported operand type(s) for -: 'tuple' and 'tuple' good answer:  643  0.003s
<< 17,845-17,202 || [] || [17845.0, 17202.0] || False >>
cac10d43fde9e07342fa7144876e77e7
What was the incr

In [27]:
annotated_results = cga_utils.annotate_results(res)
annotated_results.to_csv('res/g3_v19d_test_v2.csv')
cga_utils.calc_overall_em(annotated_results)

Annotaion warning:  invalid decimal literal (<unknown>, line 12)
Annotaion warning:  invalid decimal literal (<unknown>, line 13)


0.4437367303609342

In [17]:
import pandas as pd
messages = prompt_versions_g3.prompt_versions['V22']
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1771437733.1867845


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years?[Error]invalid syntax (<string>, line 1)
[Error]invalid syntax (<string>, line 1)
 failure: [Error]invalid syntax (<string>, line 1) good answer:  995684.5  0.003s
<< (948,578+1,042,791)/2 || [] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -8.96 good answer:  -9.03  0.004s
<< ((948,578-1,042,791)/1,042,791) * 100 || [5294879.0, 4820218.0, -0.08964529689913595] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018?[Error]unsupported operand type(s) for -: 'tuple' and 'tuple'
[Error]unsupported operand type(s) for -: 'tuple' and 'tuple'
 failure: [Error]unsupported operand type(s) for -: 'tuple' and 'tuple' good answer:  643  0.003s
<< 17,845-17,202 || [] ||

In [18]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

0.4182590233545648

In [19]:
import pandas as pd
messages = prompt_versions_g3.prompt_versions['V23b']
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1771437746.470176


0it [00:00, ?it/s]

91add58b02eb761d380b13df7a61401a
What is the average annual total assets for both Fiscal years? failure: 1166356.0 good answer:  995684.5  0.005s
<< (948,578+1,042,791)/2 || [1112987.0, 1219725.0, 1166356.0] || [948578.0, 1042791.0] || False >>
c10a228df13517c3f2312d1b281822f2
What is the percentage change of total assets from fiscal year 2018 to 2019? failure: -0.09 good answer:  -9.03  0.004s
<< ((948,578-1,042,791)/1,042,791) * 100 || [5294879.0, 4820218.0, -0.08964529689913595] || [948578.0, 1042791.0, 1042791.0] || False >>
4de9657dd64c1dc1537eae71320fd4b8
What is the difference between the Restructuring costs and other reserves in fiscal year 2019 and 2018?[Error]unsupported operand type(s) for -: 'tuple' and 'tuple'
[Error]unsupported operand type(s) for -: 'tuple' and 'tuple'
 failure: [Error]unsupported operand type(s) for -: 'tuple' and 'tuple' good answer:  643  0.003s
<< 17,845-17,202 || [] || [17845.0, 17202.0] || False >>
cac10d43fde9e07342fa7144876e77e7
What was the incr

In [20]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

Annotaion warning:  invalid decimal literal (<unknown>, line 12)
Annotaion warning:  invalid syntax (<unknown>, line 16)


0.34182590233545646